In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd

# Set credentials 
os.environ['PG_HOST'] = 'isba-dev-02.cmb4w8cmqb26.us-east-1.rds.amazonaws.com'
os.environ['PG_USER'] = 'postgres'
os.environ['PG_PASSWORD'] = 'isba_4715'
os.environ['PG_DB'] = 'data_engineer_project'

host = os.getenv('PG_HOST')
user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')
database = os.getenv('PG_DB')

connection_string = f"postgresql+psycopg2://{user}:{password}@{host}/{database}"
engine = create_engine(connection_string)
pd.set_option('display.max_rows', None)

test_query = '''
SELECT COUNT(*) FROM raw."Active_Players"
limit 10;
'''
pd.read_sql(test_query, engine)

,count
0,65
